# Load data into Pandas DataFrame

In [ ]:
import pandas as pd 

# Read into DataFrames
train = pd.read_csv("titanic/train.csv")
test = pd.read_csv("titanic/test.csv")

# At first lets check the train DataFrame
train.head(5)